# test_case_tjk_2025-08-23.ipynb


- folder for data `test_case_data` -> download separately via owncloud
- folder for configs `test_case_config`


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
import argparse
import datetime
import logging
from pathlib import Path
import numpy as np

sys.path.append('../')
import ppcpy
import ppcpy.io.loadConfigs as loadConfigs
import ppcpy.io.readPollyRawData as readPollyRawData
import ppcpy.interface.picassoProc as picassoProc
import ppcpy.misc.helper as helper

# import matplotlib
# import matplotlib.pyplot as plt
# import numpy as np
# import pprint

In [3]:
picasso_default_config_file = Path(
    helper.detect_path_type(Path.cwd().parent) ,'ppcpy','config','pollynet_processing_chain_config.json')

In [4]:
# for purpose of the notebook mimic the argparse interface
from types import SimpleNamespace
args = SimpleNamespace()

args.timestamp = '20250823'
args.device = 'pollyxt_tjk'
dt = datetime.datetime.strptime(args.timestamp, "%Y%m%d")

# the used config file
args.picasso_config_file = "test_case_config/pollynet_processing_chain_config_test.json"
# the data file to use
args.level0_file_to_process = f"test_case_data/{dt:%Y_%m_%d_%a}_TJK_00_00_01.nc"

In [5]:
picasso_config_dict = loadConfigs.loadPicassoConfig(args.picasso_config_file,picasso_default_config_file)
# print(picasso_config_dict['pollynet_config_link_file'])
polly_config_array = loadConfigs.readPollyNetConfigLinkTable(picasso_config_dict['pollynet_config_link_file'],timestamp=args.timestamp,device=args.device)
#print(polly_config_array)

polly_config_dict = loadConfigs.getPollyConfigfromArray(
    polly_config_array, picasso_config_dict
)

2026-02-13 14:51:38,978 - INFO - picasso_default_config_file: /mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/ppcpy/config/pollynet_processing_chain_config.json
2026-02-13 14:51:38,987 - INFO - picasso_config_file: test_case_config/pollynet_processing_chain_config_test.json
2026-02-13 14:51:38,996 - INFO - pollynet_config_link_file: test_case_config/pollynet_processing_chain_config_links.xlsx
2026-02-13 14:51:42,271 - INFO - polly_default_config_file: /mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/ppcpy/config/polly_global_config.json
2026-02-13 14:51:42,280 - INFO - polly_config_file: test_case_config/pollyxt_tjk_config_20231023.json
2026-02-13 14:51:42,286 - INFO - keys default/template file, but not in specific file {'Data_Originator', 'yLim_WV_RH', 'yLim_depolConst_355', 'Data_Originator_phone', 'minSNR_4_sigNorm', 'voldepol_error_532', 'yLim_FR_RCS', 'zLim_quasi_beta_532', 'channelTags', 'Data_Originator_email', 'polCaliEta1064', 'xLim_Profi_LR', 'G', 'isDFOV', 'molDepolStd1064', 'volDep

In [7]:
rawfile_fullname = args.level0_file_to_process
rawfile = helper.detect_path_type(rawfile_fullname)
rawdata_dict = readPollyRawData.readPollyRawData(rawfile)

2026-02-13 14:51:49,831 - INFO - reading nc-file: test_case_data/2025_08_23_Sat_TJK_00_00_01.nc


In [8]:
data_cube = picassoProc.PicassoProc(rawdata_dict,polly_config_dict,picasso_config_dict)

In [9]:
## reset date if date in filename differs date within nc-file 
data_cube.reset_date_infile()

## checking for correct mshots
data_cube.check_for_correct_mshots()

## setting channelTags
data_cube.setChannelTags()

## check for correct date in nc-file
data_cube.reset_date_infile()

2026-02-13 14:52:09,385 - INFO - date consistency-check... 
2026-02-13 14:52:09,388 - INFO - ... date in nc-file equals date of filename
2026-02-13 14:52:09,422 - INFO - ChannelLabels: ['FR-total-355 nm', 'FR-cross-355 nm', 'FR-387 nm', 'FR-407 nm', 'FR-total-532 nm', 'FR-cross-532 nm', 'FR-607 nm', 'FR-total-1064 nm', 'NR-total-532 nm', 'NR-607 nm', 'NR-total-355 nm', 'NR-387 nm', 'DFOV']
2026-02-13 14:52:09,426 - INFO - date consistency-check... 
2026-02-13 14:52:09,430 - INFO - ... date in nc-file equals date of filename


In [10]:
## preprocessing
data_cube.preprocessing(collect_debug=False)

2026-02-13 14:52:12,866 - INFO - starting data preprocessing...
2026-02-13 14:52:12,869 - INFO - ... time conversion
2026-02-13 14:52:12,889 - WARNING - ... mShots not constant min 2992 max 2998
2026-02-13 14:52:12,900 - INFO - ... Deadtime-correction (Mode: 3)


mShots_norm (2880, 13) mShots_norm [2995.48402778 2995.48402778 2995.48402778 2995.48402778 2995.48402778
 2995.48402778 2995.48402778 2995.48402778 2994.69097222 2994.69097222
 2994.69097222 2994.69097222 2994.69097222]


2026-02-13 14:52:43,201 - INFO - ... removing background from signal
2026-02-13 14:52:45,614 - INFO - ... height bin calculations
/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/preprocess/pollyPreprocess.py:644: UserWarning: no explicit representation of timezones available for np.datetime64
  data_dict['time64'] = np.array([np.datetime64(t) for t in mTime_obj])
2026-02-13 14:52:45,832 - INFO - ... mask bins with low SNR


flag 532 FR [False False False False  True False False False False False False False
 False]
flag 355 FR [ True False False False False False False False False False False False
 False]
flag 607 FR [False False False False False False  True False False False False False
 False]


2026-02-13 14:52:56,633 - INFO - ... mask for polarization calibration
2026-02-13 14:52:56,660 - INFO - ... calculate range-corrected Signal


(2880, 2048, 13)
ranges2d (2880, 2048)


2026-02-13 14:52:59,570 - INFO - finished data preprocessing.


In [11]:
data_cube.channel_dict

{0: 'FR-total-355 nm',
 1: 'FR-cross-355 nm',
 2: 'FR-387 nm',
 3: 'FR-407 nm',
 4: 'FR-total-532 nm',
 5: 'FR-cross-532 nm',
 6: 'FR-607 nm',
 7: 'FR-total-1064 nm',
 8: 'NR-total-532 nm',
 9: 'NR-607 nm',
 10: 'NR-total-355 nm',
 11: 'NR-387 nm',
 12: 'DFOV'}

In [12]:
data_cube.SaturationDetect()

2026-02-13 14:53:09,044 - INFO - Saturation detection


#### First bin height correctly set?

<div class="alert alert-info">
TODO: include a plot to illustrate that
</div>

In [13]:
data_cube.polarizationCaliD90()

2026-02-13 14:53:22,579 - INFO - and even a 355 channel


starting loadGHK
data_cube keys  dict_keys(['rawfile', 'rawdata_dict', 'polly_config_dict', 'picasso_config_dict', 'device', 'location', 'date', 'num_of_channels', 'num_of_profiles', 'retrievals_highres', 'retrievals_profile', 'channel_dict', 'flags', 'flag_355_total_FR', 'flag_355_cross_FR', 'flag_355_parallel_FR', 'flag_355_total_NR', 'flag_387_total_FR', 'flag_387_total_NR', 'flag_407_total_FR', 'flag_407_total_NR', 'flag_532_total_FR', 'flag_532_cross_FR', 'flag_532_parallel_FR', 'flag_532_total_NR', 'flag_532_cross_DFOV', 'flag_532_rr_FR', 'flag_607_total_FR', 'flag_607_total_NR', 'flag_1058_total_FR', 'flag_1064_total_FR', 'flag_1064_cross_FR', 'flag_1064_total_NR', 'flagSaturation'])
H is empty -> calculate parameters
TR [1.050e+00 5.000e+02 1.000e+00 1.000e+00 1.005e+00 1.666e+03 1.000e+00
 1.000e+00 1.000e+00 1.000e+00 1.000e+00 1.000e+00 1.000e+00]
G [   1.    1. -999. -999.    1.    1. -999.    1. -999. -999. -999. -999.
 -999.]
H [-2.43902439e-02 -9.96007984e-01 -9.99000000

/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/calibration/polarization.py:284: RuntimeWarning: divide by zero encountered in divide
  dplus = smooth_signal(sig_x_p, smooth_win) / smooth_signal(sig_t_p, smooth_win)
/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/calibration/polarization.py:284: RuntimeWarning: invalid value encountered in divide
  dplus = smooth_signal(sig_x_p, smooth_win) / smooth_signal(sig_t_p, smooth_win)
/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/calibration/polarization.py:285: RuntimeWarning: divide by zero encountered in divide
  dminus = smooth_signal(sig_x_m, smooth_win) / smooth_signal(sig_t_m, smooth_win)
/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/calibration/polarization.py:285: RuntimeWarning: invalid value encountered in divide
  dminus = smooth_signal(sig_x_m, smooth_win) / smooth_signal(sig_t_m, smooth_win)
2026-02-13 14:53:22,888 - INFO - pol_cali_355
2026-02-13 14:53:22,890 - INFO - and even a 53

pol_cali_nang_start_time [1755916590, 1755968190, 1755984990]
pol_cali_nang_start_time [1755916590, 1755968190, 1755984990]


2026-02-13 14:53:23,301 - INFO - pol_cali_532
2026-02-13 14:53:23,304 - WARNING - calibrateGHK no 1064 channel


In [14]:
data_cube.cloudScreen()

Starting cloud screen
cloud screen mode 1
slope_thres 4000000.0


In [15]:
data_cube.cloudFreeSeg()

intNProfiles 120 minIntNProfiles 90


In [16]:
data_cube.clFreeGrps

array([[   0,  119],
       [ 120,  300],
       [ 321,  440],
       [ 441,  560],
       [ 561,  680],
       [ 681,  800],
       [ 801,  920],
       [ 921, 1040],
       [1041, 1160],
       [1161, 1280],
       [1281, 1400],
       [1401, 1520],
       [1521, 1640],
       [1641, 1760],
       [1761, 1880],
       [1881, 2020],
       [2041, 2160],
       [2161, 2280],
       [2281, 2400],
       [2401, 2580],
       [2601, 2720],
       [2721, 2879]])

In [17]:
# data_cube.clFreeGrps = [
#     [35, 300],
#     [1000, 1300],
#     [2650, 2870]
# ]

In [18]:
data_cube.aggregate_profiles()

In [19]:
data_cube.loadMeteo()

In [20]:
data_cube.calcMolecular()

time slices of cloud free  [array(['2025-08-23T00:00:00.000000', '2025-08-23T00:59:30.000000'],
      dtype='datetime64[us]'), array(['2025-08-23T01:00:00.000000', '2025-08-23T02:30:00.000000'],
      dtype='datetime64[us]'), array(['2025-08-23T02:40:30.000000', '2025-08-23T03:40:00.000000'],
      dtype='datetime64[us]'), array(['2025-08-23T03:40:30.000000', '2025-08-23T04:40:00.000000'],
      dtype='datetime64[us]'), array(['2025-08-23T04:40:30.000000', '2025-08-23T05:40:00.000000'],
      dtype='datetime64[us]'), array(['2025-08-23T05:40:30.000000', '2025-08-23T06:40:00.000000'],
      dtype='datetime64[us]'), array(['2025-08-23T06:40:30.000000', '2025-08-23T07:40:00.000000'],
      dtype='datetime64[us]'), array(['2025-08-23T07:40:30.000000', '2025-08-23T08:40:00.000000'],
      dtype='datetime64[us]'), array(['2025-08-23T08:40:30.000000', '2025-08-23T09:40:00.000000'],
      dtype='datetime64[us]'), array(['2025-08-23T09:40:30.000000', '2025-08-23T10:40:00.000000'],
      dtype='

In [21]:
data_cube.rayleighFit()

2026-02-13 14:53:55,552 - WARNING - Potential for differences to matlab code du to numerical issues (subtraction of two small values)
2026-02-13 14:53:55,556 - WARNING - rayleighfit seems to use range in matlab, but the met data should be in height >> RECHECK!
2026-02-13 14:53:55,557 - WARNING - at 10km height this is a difference of about 4 indices


Start Rayleigh Fit
0 [  0 119]
refH for 532
refHInd (773, 973)
refH for 355
refHInd (772, 972)
refH for 1064
refHInd (798, 998)
1 [120 300]
refH for 532
one tests failed?
refHInd (767, 967)
refH for 355


/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/calibration/rayleighfit.py:403: RuntimeWarning: divide by zero encountered in divide
  std_aer_norm = sig_aer_norm / np.sqrt(pc + bg)


one tests failed?
refHInd (758, 958)
refH for 1064
one tests failed?
refHInd (827, 1027)
2 [321 440]
refH for 532
refHInd (801, 898)
refH for 355
one tests failed?
refHInd (790, 990)
refH for 1064
one tests failed?
one tests failed?
refHInd (nan, nan)
3 [441 560]
refH for 532
one tests failed?
refHInd (nan, nan)
refH for 355
one tests failed?
refHInd (nan, nan)
refH for 1064
one tests failed?
one tests failed?
refHInd (1133, 1230)
4 [561 680]
refH for 532
one tests failed?
refHInd (nan, nan)
refH for 355
one tests failed?
refHInd (nan, nan)
refH for 1064
one tests failed?
one tests failed?
refHInd (nan, nan)
5 [681 800]
refH for 532
refHInd (nan, nan)
refH for 355
refHInd (nan, nan)
refH for 1064
refHInd (1019, 1114)
6 [801 920]
refH for 532
refHInd (nan, nan)
refH for 355
refHInd (nan, nan)
refH for 1064
refHInd (nan, nan)
7 [ 921 1040]
refH for 532
refHInd (nan, nan)
refH for 355
refHInd (nan, nan)
refH for 1064
one tests failed?
refHInd (nan, nan)
8 [1041 1160]
refH for 532
refHInd 

[{'532_total_FR': {'DPInd': array([  86,  286,  287,  401,  469,  669,  670,  676,  773,  973,  974,
          1174, 1175, 1318]),
   'refHInd': (773, 973)},
  '355_total_FR': {'DPInd': array([  86,  286,  287,  487,  488,  670,  772,  972,  973, 1173, 1174,
          1318]),
   'refHInd': (772, 972)},
  '1064_total_FR': {'DPInd': array([ 116,  316,  317,  385,  494,  668,  730,  779,  798,  998,  999,
          1199, 1200, 1288]),
   'refHInd': (798, 998)}},
 {'532_total_FR': {'DPInd': array([  86,  286,  287,  372,  476,  676,  677,  703,  767,  967,  968,
          1168, 1169, 1318]),
   'refHInd': (767, 967)},
  '355_total_FR': {'DPInd': array([  86,  286,  287,  487,  488,  688,  689,  702,  758,  958,  959,
          1159, 1160, 1318]),
   'refHInd': (758, 958)},
  '1064_total_FR': {'DPInd': array([ 116,  316,  317,  517,  518,  682,  746,  788,  827, 1027, 1028,
          1228, 1229, 1288]),
   'refHInd': (827, 1027)}},
 {'532_total_FR': {'DPInd': array([  86,  286,  287,  418, 

In [22]:
# Use config values for refH in NR channels (similar approche to Picasso)
for e in data_cube.refH:
    e['532_total_NR'] = {'DPInd':None, 'refHInd':tuple(np.searchsorted(data_cube.retrievals_highres["height"], data_cube.polly_config_dict["refH_NR_532"]) - [0, 1])}     # - [0, 1] --> make sure all values lay inside the designeted range
    e['355_total_NR'] = {'DPInd':None, 'refHInd':tuple(np.searchsorted(data_cube.retrievals_highres["height"], data_cube.polly_config_dict["refH_NR_355"]) - [0, 1])}     # - [0, 1] --> make sure all values lay inside the designeted range

In [23]:
data_cube.transCor()

2026-02-13 14:54:06,933 - WARNING - transmission correction
2026-02-13 14:54:07,677 - INFO - and even a 355 channel


G [1.] [1.]
H [-0.02439024] [-0.99600798]
polCaliEta 10.754878969562773
G [1.] [1.] H [-0.02439024] [-0.99600798] Eta 10.754878969562773 error [0.003 0.    0.   ] Window 1 


/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/retrievals/depolarization.py:161: RuntimeWarning: divide by zero encountered in divide
  sig_ratio = sigc / sigt
/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/retrievals/depolarization.py:161: RuntimeWarning: invalid value encountered in divide
  sig_ratio = sigc / sigt
/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/retrievals/depolarization.py:171: RuntimeWarning: invalid value encountered in divide
  vol_depol = (sig_ratio / eta * (Gt + Ht) - (Gr + Hr)) / ((Gr - Hr) - sig_ratio / eta * (Gt - Ht))
2026-02-13 14:54:08,411 - INFO - and even a 532 channel


calculated R_t [1.05]
G [1.] [1.]
H [-0.00249377] [-0.99880024]
polCaliEta 6.852638549685832
G [1.] [1.] H [-0.00249377] [-0.99880024] Eta 6.852638549685832 error [0.004 0.    0.   ] Window 1 
calculated R_t [1.005]


In [24]:
data_cube.aggregate_profiles(var='sigTCor')
data_cube.aggregate_profiles(var='BGTCor')

In [25]:
data_cube.retrievalKlett(nr=True)

2026-02-13 14:54:18,873 - WARNING - rayleighfit seems to use range in matlab, but the met data should be in height >> RECHECK!
2026-02-13 14:54:18,879 - WARNING - at 10km height this is a difference of about 4 indices


retrievalname klett
Starting Klett retrieval
cldFree  0 [  0 119]
cldFree mod (np.int64(0), np.int64(120))
== 532, total, FR klett =================================
refHInd (773, 973) refH [5781.92492628 7276.92490721]
LR  40 refH [5781.92492628 7276.92490721] refBeta 1e-08 smoothWin_klett 21
== 355, total, FR klett =================================
refHInd (772, 972) refH [5774.44992638 7269.4499073 ]
LR  45 refH [5774.44992638 7269.4499073 ] refBeta 2e-08 smoothWin_klett 21
== 1064, total, FR klett =================================
refHInd (798, 998) refH [5968.7999239  7463.79990482]
LR  50 refH [5968.7999239  7463.79990482] refBeta 2e-09 smoothWin_klett 21
== 532, total, NR klett =================================
refHInd (np.int64(336), np.int64(402)) refH [2515.34996796 3008.69996166]
LR  50 refH [2515.34996796 3008.69996166] refBeta 2.288463648378021e-06 smoothWin_klett 13
== 355, total, NR klett =================================
refHInd (np.int64(336), np.int64(402)) refH [2515.

In [26]:
data_cube.retrievalRaman(nr=True)

2026-02-13 14:54:24,183 - WARNING - rayleighfit seems to use range in matlab, but the met data should be in height >> RECHECK!
2026-02-13 14:54:24,188 - WARNING - at 10km height this is a difference of about 4 indices


[array([500, 500, 500, 500, 500, 500, 500, 500, 150, 150, 150, 150, 150]), array([500, 500, 500, 500, 500, 500, 500, 500, 150, 150, 150, 150, 150]), array([500, 500, 500, 500, 500, 500, 500, 500, 150, 150, 150, 150, 150]), array([500, 500, 500, 500, 500, 500, 500, 500, 150, 150, 150, 150, 150]), array([500, 500, 500, 500, 500, 500, 500, 500, 150, 150, 150, 150, 150]), array([500, 500, 500, 500, 500, 500, 500, 500, 150, 150, 150, 150, 150]), array([500, 500, 500, 500, 500, 500, 500, 500, 150, 150, 150, 150, 150]), array([500, 500, 500, 500, 500, 500, 500, 500, 150, 150, 150, 150, 150]), array([500, 500, 500, 500, 500, 500, 500, 500, 150, 150, 150, 150, 150]), array([500, 500, 500, 500, 500, 500, 500, 500, 150, 150, 150, 150, 150]), array([500, 500, 500, 500, 500, 500, 500, 500, 150, 150, 150, 150, 150]), array([500, 500, 500, 500, 500, 500, 500, 500, 150, 150, 150, 150, 150]), array([500, 500, 500, 500, 500, 500, 500, 500, 150, 150, 150, 150, 150]), array([500, 500, 500, 500, 500, 500, 

/mnt/c/Users/radenz/dev/PicassoPy/pyppc_env/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:2053: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/retrievals/raman.py:494: RuntimeWarning: invalid value encountered in sqrt
  sigElasticSample = sigGenWithNoise(sigElastic, np.sqrt(sigElastic + bgElastic), MC_count[2], 'norm').T


500 227.9875
refHInd (772, 972) refH [5774.44992638 7269.4499073 ] hBaseInd 97 hBase 728.8249907493591
filling aerExt below overlap with 3.374759565323807e-05 for calculating the backscatter
== 532, total, FR | 607, total, FR raman ========
500 227.9875
refHInd (773, 973) refH [5781.92492628 7276.92490721] hBaseInd 97 hBase 728.8249907493591
filling aerExt below overlap with -0.0001718884383956104 for calculating the backscatter
== 1064, total, FR | 607, total, FR raman ========
500 227.9875
refHInd (798, 998) refH [5968.7999239  7463.79990482] hBaseInd 97 hBase 728.8249907493591
filling aerExt below overlap with -0.00013961679599276572 for calculating the backscatter
== 532, total, NR | 607, total, NR raman ========
150 93.4375
refHInd (np.int64(336), np.int64(402)) refH [2515.34996796 3008.69996166] hBaseInd 33 hBase 250.42499685287476
filling aerExt below overlap with 0.0002816784198640547 for calculating the backscatter
== 355, total, NR | 387, total, NR raman ========
150 93.4375


### skipped the rest of the optical profiles and the OL correction for now

In [27]:
data_cube.LidarCalibration()

retival klett
LCMeanWindow 50 LCMeanMinIndx 69 LCMeanMaxIndx 999
cldFreGrp 0, Channel 532 total FR, LC_stable 76651241245552.4, LCStd 0.0005007734961363295
cldFreGrp 0, Channel 355 total FR, LC_stable 61221519590893.43, LCStd 0.00024937033480927216
cldFreGrp 0, Channel 1064 total FR, LC_stable 513592389824038.4, LCStd 0.0005981479464759632
cldFreGrp 0, Channel 532 total NR, LC_stable 7574738147850.264, LCStd 0.0007231192269635228
cldFreGrp 0, Channel 355 total NR, LC_stable 9735990759832.596, LCStd 0.0003160177325644267
cldFreGrp 1, Channel 532 total FR, LC_stable 79019754266917.53, LCStd 0.0002538674788494131
cldFreGrp 1, Channel 355 total FR, LC_stable 62415842599511.72, LCStd 0.0003252469056711153
cldFreGrp 1, Channel 1064 total FR, LC_stable 517684901664567.7, LCStd 0.0003097064771886713
cldFreGrp 1, Channel 532 total NR, LC_stable 7729289211025.326, LCStd 0.0005540578799784854
cldFreGrp 1, Channel 355 total NR, LC_stable 9716113458500.895, LCStd 0.00029344913514387844
cldFreGrp 2,

/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/misc/helper.py:758: RuntimeWarning: Mean of empty slice
  thisMean = np.nanmean(window)


cldFreGrp 0, Channel 387 total FR, LC_stable 80027915768939.3, LCStd 0.0003038744980764333
cldFreGrp 0, Channel 532 total FR, LC_stable 45401180463282.01, LCStd 0.0011639818900432413
cldFreGrp 0, Channel 607 total FR, LC_stable 94197550133957.02, LCStd 0.0006115797907810598
cldFreGrp 0, Channel 1064 total FR, LC_stable 398981493960142.2, LCStd 0.0009619087529983695
cldFreGrp 0, Channel 532 total NR, LC_stable 7130550543865.869, LCStd 0.0014434875818810026
cldFreGrp 0, Channel 607 total NR, LC_stable 19209931756656.742, LCStd 0.0012191819005460812
cldFreGrp 0, Channel 355 total NR, LC_stable 11052820552651.648, LCStd 0.0007228416002316402
cldFreGrp 0, Channel 387 total NR, LC_stable 19399243104969.566, LCStd 0.0008591807913631008
cldFreGrp 1, Channel 355 total FR, LC_stable 56119594740424.71, LCStd 0.0011674689200523649
cldFreGrp 1, Channel 387 total FR, LC_stable 80170738326909.72, LCStd 0.0002651927324375641
cldFreGrp 1, Channel 532 total FR, LC_stable 42042794048474.89, LCStd 0.00156

2026-02-13 14:55:34,882 - WARNING - reading calibration constant from database not working yet


cldFreGrp 21, Channel 387 total NR, LC_stable 18800599072943.656, LCStd 0.0011782752347452268


In [28]:
data_cube.LCused

{'532_total_FR': np.float64(47108306455515.89),
 '532_total_NR': np.float64(6648908941932.83),
 '355_total_FR': np.float64(51989187290529.45),
 '355_total_NR': np.float64(9549656392339.236),
 '1064_total_FR': np.float64(-1.2149426389455192e+16),
 '607_total_NR': np.float64(19392045322261.797),
 '387_total_NR': np.float64(19399243104969.566),
 '607_total_FR': np.float64(105715573982426.38),
 '387_total_FR': np.float64(80170738326909.72)}

#### Result of `data_cube.LCused`

```
{'355_total_NR': np.float64(9553974818526.242),
 '355_total_FR': np.float64(51988705902924.55),
 '532_total_FR': np.float64(47108560396460.03),
 '1064_total_FR': np.float64(-1.2149426389455192e+16),
 '532_total_NR': np.float64(6645236078492.683),
 '387_total_NR': np.float64(19399243104969.566),
 '607_total_NR': np.float64(19392045322261.797),
 '607_total_FR': np.float64(105715573982426.38),
 '387_total_FR': np.float64(80170738326909.72)}
```

<div class="alert alert-warning">
The negative LC in 1064 should be incorrect, check later
</div>